In [3]:
# Some of the codes will be very similar to our previous code from coursework 1.
# Importing all the necessary packages

import pandas as pd # For data reading
import chardet # For detecting the correct encoding of the data
import re # For preprocessing
from nltk.corpus import stopwords # For removing stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import torch
import torch.nn as nn # for perceptron classifier
import torch.optim as optim # to optimise the loss function


# Downloading
import nltk
nltk.download('punkt') #tokenizer
nltk.download('stopwords') # stopwords
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
# Loading the data (Note that the file was not encoded in uft8 hence why we need to find the encoding and set it to that when using pd.read_csv)

# First we detect encoding using chardet
with open("Semeval.txt", "rb") as f:
    result = chardet.detect(f.read())
Encoding = result['encoding'] # store the coding into the variable Encoding
print(f"Detected encoding: {Encoding}") # It was encoded in Windows-1252

# Next we read the file with the correct encoding
Train_data = pd.read_csv("Semeval.txt", delimiter="\t", encoding=Encoding)

# Let's display the top rows.
print(Train_data.head())


#Identifying the columns

Id = Train_data['ID']
Target = Train_data['Target']
Tweet = Train_data['Tweet']
Stance = Train_data['Stance']

Detected encoding: Windows-1252
    ID   Target                                              Tweet   Stance
0  101  Atheism  dear lord thank u for all of ur blessings forg...  AGAINST
1  102  Atheism  Blessed are the peacemakers, for they shall be...  AGAINST
2  103  Atheism  I am not conformed to this world. I am transfo...  AGAINST
3  104  Atheism  Salah should be prayed with #focus and #unders...  AGAINST
4  105  Atheism  And stay in your houses and do not display you...  AGAINST


In [5]:
# Preprocess

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to preprocess tweets
def preprocess_tweet(tweet):
    # Lowercase
    tweet = tweet.lower()

    # Remove mentions
    tweet = re.sub(r"@\w+", "", tweet)

    # Remove hashtags
    tweet = re.sub(r"#\w+", "", tweet)

    # Remove punctuation
    tweet = re.sub(r"[^\w\s]", "", tweet)

    # Remove numbers
    tweet = re.sub(r"\d+", "", tweet)

    # Tokenize
    words = word_tokenize(tweet)

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Join tokens back into a string
    tweet = " ".join(tokens)

    return tweet

# We apply preprocessing to the 'Tweet' column
Train_data['Tweet'] = Train_data['Tweet'].apply(preprocess_tweet)

# Display the first few rows
print(Train_data.head())

    ID   Target                                              Tweet   Stance
0  101  Atheism  dear lord thank u ur blessing forgive sin lord...  AGAINST
1  102  Atheism  blessed peacemaker shall called child god matthew  AGAINST
2  103  Atheism          conformed world transformed renewing mind  AGAINST
3  104  Atheism       salah prayed warns prayer done surah almaoon  AGAINST
4  105  Atheism       stay house display like time ignorance quran  AGAINST


In [6]:
# Loading the test data
"""
We will load the test data and we preprocess it.
"""

# Detecing the code for the test data
with open("Testdata.txt", "rb") as f:
    result = chardet.detect(f.read())
Encoding = result['encoding'] # store the coding into the variable Encoding
print(f"Detected encoding: {Encoding}") # It was encoded in Windows-1252

# Reading the test data with the right encoding
Test_data = pd.read_csv("Testdata.txt", delimiter="\t", encoding=Encoding)

# Let's display the top rows.
print(Test_data.head())


# Preprocessing
Test_data['Tweet'] = Test_data['Tweet'].apply(preprocess_tweet)

#Identifying the columns for the test data

Id_test = Test_data['ID']
Target_test = Test_data['Target']
Tweet_test = Test_data['Tweet']
Stance_test = Test_data['Stance']



Detected encoding: ascii
      ID   Target                                              Tweet   Stance
0  10001  Atheism  He who exalts himself shall      be humbled; a...  AGAINST
1  10002  Atheism  RT @prayerbullets: I remove Nehushtan -previou...  AGAINST
2  10003  Atheism  @Brainman365 @heidtjj @BenjaminLives I have so...  AGAINST
3  10004  Atheism  #God is utterly powerless without Human interv...  AGAINST
4  10005  Atheism  @David_Cameron   Miracles of #Multiculturalism...  AGAINST


In [7]:
#Word embedding GLOVE

glove_embeddings = {} # An empty dictionary where we will store word-vector pairs

with open("glove.6B.300d.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        glove_embeddings[word] = vector




# Function to get sentence embeddings
def get_sentence_embedding_Glove(sentence, embeddings):
    words = sentence.lower().split()
    vectors = [embeddings[word] for word in words if word in embeddings]
    if len(vectors) == 0:
        return np.zeros(300)  # Return zero vector if no words are found
    return np.mean(vectors, axis=0)

In [6]:
# Turn text column into vector

X_train = np.array([get_sentence_embedding_Glove(tweet, glove_embeddings) for tweet in Tweet])
X_test = np.array([get_sentence_embedding_Glove(tweet, glove_embeddings) for tweet in Tweet_test])

In [7]:
from torch.utils.data import TensorDataset, DataLoader

# pytorch tensor conversion
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(Stance.factorize()[0], dtype=torch.long)
y_test_tensor = torch.tensor(Stance_test.factorize()[0], dtype=torch.long)

# Create TensorDataset (hybrid approach)
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
# Feed forward Neural Network Model

class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize model
INPUT_DIM = 300  # GloVe vector size
HIDDEN_DIM = 128
OUTPUT_DIM = 3    # FAVOR, AGAINST, NONE
model = FeedforwardNN(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)


In [9]:
# Training Setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# training loop

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss/len(train_loader):.4f}')

Epoch 1/20, Loss: 1.0456
Epoch 2/20, Loss: 1.0011
Epoch 3/20, Loss: 0.9699
Epoch 4/20, Loss: 0.9455
Epoch 5/20, Loss: 0.9218
Epoch 6/20, Loss: 0.9039
Epoch 7/20, Loss: 0.8864
Epoch 8/20, Loss: 0.8693
Epoch 9/20, Loss: 0.8517
Epoch 10/20, Loss: 0.8379
Epoch 11/20, Loss: 0.8137
Epoch 12/20, Loss: 0.8020
Epoch 13/20, Loss: 0.7847
Epoch 14/20, Loss: 0.7738
Epoch 15/20, Loss: 0.7519
Epoch 16/20, Loss: 0.7528
Epoch 17/20, Loss: 0.7270
Epoch 18/20, Loss: 0.7132
Epoch 19/20, Loss: 0.7029
Epoch 20/20, Loss: 0.6806


In [11]:
# Evaluation

from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, data_loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, f1

test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nTest Accuracy: {test_acc:.4f}, Test F1-score: {test_f1:.4f}')


Test Accuracy: 0.5212, Test F1-score: 0.5365


In [12]:
# Improvement 1: class imbalance

# Calculate inverse frequency weights
class_counts = np.array([0.26, 0.48, 0.26])  # FAVOR : 0.26, AGAINST : 0.48, NONE : 0.26
weights = torch.tensor(class_counts.max() / class_counts, dtype=torch.float32).to(device)
# Result: weights ≈ [1.85, 1.0, 1.85]

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()



test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nTest Accuracy: {test_acc:.4f}, Test F1-score: {test_f1:.4f}')


Test Accuracy: 0.5132, Test F1-score: 0.5297


In [13]:
# Improvement 2: Deeper architecture

class EnhancedNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)  # 3 output classes
        self.dropout = nn.Dropout(0.5)  # Add dropout
        self.batchnorm1 = nn.BatchNorm1d(256)  # Add batch norm

    def forward(self, x):
        x = self.dropout(torch.relu(self.batchnorm1(self.fc1(x))))
        x = self.dropout(torch.relu(self.fc2(x)))
        return self.fc3(x)

# Initialize model
model = EnhancedNN(input_dim=300)


# Training Setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# training loop

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()


test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nTest Accuracy: {test_acc:.4f}, Test F1-score: {test_f1:.4f}')



Test Accuracy: 0.5268, Test F1-score: 0.5420


In [14]:
# Improvement 3: Changing mean to sum for better embedding

def text_to_vector(text, embedding_model):
    words = text.split()
    vectors = [embedding_model[word] for word in words if word in embedding_model]
    # Check if vectors is not empty, then get the size from the first vector
    return np.sum(vectors, axis=0) if vectors else np.zeros(len(embedding_model[next(iter(embedding_model))]))

X_train = np.array([text_to_vector(tweet, glove_embeddings) for tweet in Tweet])
y_train = Stance.factorize()[0]
X_test = np.array([text_to_vector(tweet, glove_embeddings) for tweet in Tweet_test])
y_test = Stance_test.factorize()[0]

# Normalize (critical for sum embeddings!)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train)),
                         batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test)),
                        batch_size=32, shuffle=False)

# Initialize model
model = FeedforwardNN(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)

# Training Setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

#Evaluation
test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nTest Accuracy: {test_acc:.4f}, Test F1-score: {test_f1:.4f}')


Test Accuracy: 0.4940, Test F1-score: 0.5092


In [15]:
# Improvement 4: Enhanced NN + Changing mean to sum for better embedding


X_train = np.array([text_to_vector(tweet, glove_embeddings) for tweet in Tweet])
y_train = Stance.factorize()[0]
X_test = np.array([text_to_vector(tweet, glove_embeddings) for tweet in Tweet_test])
y_test = Stance_test.factorize()[0]

# Normalize (critical for sum embeddings!)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train)),
                         batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test)),
                        batch_size=32, shuffle=False)

# Initialize model
model = EnhancedNN(input_dim=300)

# Training Setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

#Evaluation
test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nTest Accuracy: {test_acc:.4f}, Test F1-score: {test_f1:.4f}')


Test Accuracy: 0.5420, Test F1-score: 0.5526


In [16]:
#Improvment 5: optimised enhanced nn for sum embedding + less aggresive weights

class EnhancedNN_SumSafe(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)
        self.dropout = nn.Dropout(0.3)  # Reduced dropout
        # REMOVE BatchNorm for sum embeddings!

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# 1. Initialize sum-safe model
model = EnhancedNN_SumSafe(input_dim=300).to(device)

# 2. Use milder class weights (avoid over-correction)
class_weights = torch.tensor([1.5, 1.0, 1.3]).to(device)  # Less aggressive
criterion = nn.CrossEntropyLoss(weight=class_weights)

# 3. Smaller learning rate for summed embeddings
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

# 4. Gradient clipping to control exploding magnitudes
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# training loop

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

#Evaluation
test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nTest Accuracy: {test_acc:.4f}, Test F1-score: {test_f1:.4f}')


Test Accuracy: 0.5324, Test F1-score: 0.5413


In [17]:
# Sentence-BERT for FNN

from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder

# Load SBERT (384-dimensional embeddings)
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all tweets
X_train = sbert.encode(Train_data['Tweet'].tolist(), show_progress_bar=True)
X_test = sbert.encode(Test_data['Tweet'].tolist(), show_progress_bar=True)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(Train_data['Stance'])
y_test = le.transform(Test_data['Stance'])

# Normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create DataLoaders
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

class EnhancedNN_SBERT(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(384, 256)  # SBERT dim=384
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)
        self.dropout = nn.Dropout(0.3)
        self.batchnorm1 = nn.BatchNorm1d(256)  # Safe with SBERT's normalized outputs

    def forward(self, x):
        x = self.dropout(torch.relu(self.batchnorm1(self.fc1(x))))
        x = self.dropout(torch.relu(self.fc2(x)))
        return self.fc3(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedNN_SBERT().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)

# Training
EPOCHS = 15
best_f1 = 0
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        epoch_loss += loss.item()

    # Validation
    val_acc, val_f1 = evaluate(model, test_loader)
    scheduler.step(val_f1)

    print(f'Epoch {epoch+1}/{EPOCHS} | Loss: {epoch_loss/len(train_loader):.4f} | Val F1: {val_f1:.4f}')

    # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_sbert_model.pt')

# Load and test best model
model.load_state_dict(torch.load('best_sbert_model.pt'))
test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nFinal Test Accuracy: {test_acc:.4f}, F1: {test_f1:.4f}')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/88 [00:00<?, ?it/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1/15 | Loss: 0.9789 | Val F1: 0.5966
Epoch 2/15 | Loss: 0.8429 | Val F1: 0.5968
Epoch 3/15 | Loss: 0.7638 | Val F1: 0.5903
Epoch 4/15 | Loss: 0.6874 | Val F1: 0.5961
Epoch 5/15 | Loss: 0.6085 | Val F1: 0.5833
Epoch 6/15 | Loss: 0.5082 | Val F1: 0.5955
Epoch 7/15 | Loss: 0.4990 | Val F1: 0.5896
Epoch 8/15 | Loss: 0.4928 | Val F1: 0.5851
Epoch 9/15 | Loss: 0.4687 | Val F1: 0.5852
Epoch 10/15 | Loss: 0.4674 | Val F1: 0.5865
Epoch 11/15 | Loss: 0.4676 | Val F1: 0.5922
Epoch 12/15 | Loss: 0.4718 | Val F1: 0.5920
Epoch 13/15 | Loss: 0.4742 | Val F1: 0.5939
Epoch 14/15 | Loss: 0.4665 | Val F1: 0.5867
Epoch 15/15 | Loss: 0.4678 | Val F1: 0.5858

Final Test Accuracy: 0.6077, F1: 0.5968


In [18]:
# Sentence-BERT 768D

sbert = SentenceTransformer('all-mpnet-base-v2')  # 768-dim

X_train = sbert.encode(Train_data['Tweet'].tolist(), show_progress_bar=True)
X_test = sbert.encode(Test_data['Tweet'].tolist(), show_progress_bar=True)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(Train_data['Stance'])
y_test = le.transform(Test_data['Stance'])

# Normalize (critical for SBERT)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create DataLoaders
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

class EnhancedNN_SBERT_768(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 256)  # SBERT dim=768
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)
        self.dropout = nn.Dropout(0.3)
        self.batchnorm1 = nn.BatchNorm1d(256)

    def forward(self, x):
        x = self.dropout(torch.relu(self.batchnorm1(self.fc1(x))))
        x = self.dropout(torch.relu(self.fc2(x)))
        return self.fc3(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedNN_SBERT_768().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)

# Training
EPOCHS = 15
best_f1 = 0
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        epoch_loss += loss.item()

    # Validation
    val_acc, val_f1 = evaluate(model, test_loader)
    scheduler.step(val_f1)

    print(f'Epoch {epoch+1}/{EPOCHS} | Loss: {epoch_loss/len(train_loader):.4f} | Val F1: {val_f1:.4f}')

    # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_sbert_model.pt')

# Load and test best model
model.load_state_dict(torch.load('best_sbert_model.pt'))
test_acc, test_f1 = evaluate(model, test_loader)
print(f'\nFinal Test Accuracy: {test_acc:.4f}, F1: {test_f1:.4f}')



Batches:   0%|          | 0/88 [00:00<?, ?it/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1/15 | Loss: 0.9446 | Val F1: 0.6300
Epoch 2/15 | Loss: 0.7979 | Val F1: 0.6102
Epoch 3/15 | Loss: 0.7050 | Val F1: 0.6101
Epoch 4/15 | Loss: 0.6138 | Val F1: 0.6115
Epoch 5/15 | Loss: 0.4972 | Val F1: 0.6027
Epoch 6/15 | Loss: 0.4765 | Val F1: 0.6035
Epoch 7/15 | Loss: 0.4467 | Val F1: 0.5985
Epoch 8/15 | Loss: 0.4333 | Val F1: 0.6023
Epoch 9/15 | Loss: 0.4194 | Val F1: 0.6038
Epoch 10/15 | Loss: 0.4294 | Val F1: 0.6045
Epoch 11/15 | Loss: 0.4296 | Val F1: 0.6043
Epoch 12/15 | Loss: 0.4187 | Val F1: 0.6054
Epoch 13/15 | Loss: 0.4198 | Val F1: 0.6080
Epoch 14/15 | Loss: 0.4221 | Val F1: 0.6058
Epoch 15/15 | Loss: 0.4195 | Val F1: 0.6017

Final Test Accuracy: 0.6309, F1: 0.6300


In [23]:
# Full BERT

import torch
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3,  # FAVOR, AGAINST, NONE
    output_attentions=False,
    output_hidden_states=False
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Dataset class

class BertStanceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [25]:
# Data preparation

# Convert labels to numerical (FAVOR:0, AGAINST:1, NONE:2)
label_map = {'FAVOR': 0, 'AGAINST': 1, 'NONE': 2}
y_train = [label_map[stance] for stance in Train_data['Stance']]
y_test = [label_map[stance] for stance in Test_data['Stance']]

# Create datasets
train_dataset = BertStanceDataset(
    texts=Train_data['Tweet'].tolist(),
    labels=y_train,
    tokenizer=tokenizer
)
test_dataset = BertStanceDataset(
    texts=Test_data['Tweet'].tolist(),
    labels=y_test,
    tokenizer=tokenizer
)

# DataLoaders
BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [26]:
# Training Loop

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

EPOCHS = 3
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}')

Epoch 1/3, Loss: 1.0101
Epoch 2/3, Loss: 0.8174
Epoch 3/3, Loss: 0.5835


In [29]:
print(model)
torch.save(model.state_dict(), "/content/bert_stance_detection.pt")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [30]:
from google.colab import files
files.download("/content/bert_stance_detection.pt")  # Manual download

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# Evaluation

def evaluate(model, data_loader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return accuracy_score(true_labels, predictions), \
           f1_score(true_labels, predictions, average='weighted')

accuracy, f1 = evaluate(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}, F1: {f1:.4f}')

Test Accuracy: 0.6261, F1: 0.6024


In [14]:
# CNN

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def text_to_sequence(text, max_len=50):
    words = text.lower().split()[:max_len]
    sequence = []
    for word in words:
        if word in glove_embeddings:
            sequence.append(glove_embeddings[word])
        else:
            sequence.append(np.zeros(300))
    # Pad with zeros if shorter than max_len
    while len(sequence) < max_len:
        sequence.append(np.zeros(300))
    return np.array(sequence)  # Shape: (max_len, 300)

# Convert all tweets
X_train_seq = np.array([text_to_sequence(tweet) for tweet in Train_data['Tweet']])
X_test_seq = np.array([text_to_sequence(tweet) for tweet in Test_data['Tweet']])

# Convert stance labels to numerical values (FAVOR:0, AGAINST:1, NONE:2)
label_map = {'FAVOR': 0, 'AGAINST': 1, 'NONE': 2}
y_train = np.array([label_map[stance] for stance in Train_data['Stance']])
y_test = np.array([label_map[stance] for stance in Test_data['Stance']])



# 3. Define CNN Model
class TextCNN(nn.Module):
    def __init__(self, embed_dim=300, num_filters=100, filter_sizes=[1,2]):  # Changed to [1,2]
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embed_dim))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(num_filters * len(filter_sizes), 3)

    def forward(self, x):
        x = x.unsqueeze(1)  # Shape: (batch, 1, seq_len, embed_dim)
        # For filter_size=1, this acts like "unigrams"
        features = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]
        pooled = [torch.max_pool1d(feat, feat.size(2)).squeeze(2) for feat in features]
        combined = torch.cat(pooled, 1)
        return self.fc(combined)


# Convert to PyTorch tensors
train_dataset = TensorDataset(torch.FloatTensor(X_train_seq), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.FloatTensor(X_test_seq), torch.LongTensor(y_test))

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# 4. Initialize and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch.to(device))
        loss = criterion(outputs, y_batch.to(device))
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# 5. Evaluate
def evaluate(model, data_loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            outputs = model(X_batch.to(device))
            _, preds = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, f1

accuracy, f1 = evaluate(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}, F1: {f1:.4f}')

Epoch 1, Loss: 0.8714
Epoch 2, Loss: 0.8925
Epoch 3, Loss: 0.9044
Epoch 4, Loss: 0.6704
Epoch 5, Loss: 0.5419
Epoch 6, Loss: 0.3214
Epoch 7, Loss: 0.2305
Epoch 8, Loss: 0.2383
Epoch 9, Loss: 0.1803
Epoch 10, Loss: 0.1303
Test Accuracy: 0.5677, F1: 0.5633


In [15]:
# Improved CNN with adjusted weights and larger filter

class ImprovedTextCNN(nn.Module):
    def __init__(self, embed_dim=300, num_filters=100, filter_sizes=[2,3,4]):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embed_dim))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(num_filters * len(filter_sizes), 3)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch, 1, seq_len, embed_dim)
        features = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]
        pooled = [torch.max_pool1d(feat, feat.size(2)).squeeze(2) for feat in features]
        combined = torch.cat(pooled, 1)
        return self.fc(combined)

# Based on our class distribution (FAVOR:26%, AGAINST:48%, NONE:26%)
weights = torch.tensor([1.85, 1.0, 1.85]).to(device)  # Penalize minority classes

# 4. Initialize and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedTextCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=weights)

for epoch in range(10):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch.to(device))
        loss = criterion(outputs, y_batch.to(device))
        loss.backward()
        optimizer.step()

accuracy, f1 = evaluate(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}, F1: {f1:.4f}')

Test Accuracy: 0.5348, F1: 0.5458


In [16]:
# Improvment wihtout adjusting weights

# 4. Initialize and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedTextCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch.to(device))
        loss = criterion(outputs, y_batch.to(device))
        loss.backward()
        optimizer.step()

accuracy, f1 = evaluate(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}, F1: {f1:.4f}')

Test Accuracy: 0.5653, F1: 0.5648


In [17]:
# Adding dropout

class ImprovedDropoutTextCNN(nn.Module):
    def __init__(self, embed_dim=300, num_filters=100, filter_sizes=[2,3,4], dropout_rate=0.5):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embed_dim))
            for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(dropout_rate)  # Dropout layer
        self.fc = nn.Linear(num_filters * len(filter_sizes), 3)

    def forward(self, x):
        x = x.unsqueeze(1)  # Shape: (batch, 1, seq_len, embed_dim)
        # Convolution + ReLU + MaxPool
        features = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]
        pooled = [torch.max_pool1d(feat, feat.size(2)).squeeze(2) for feat in features]
        combined = torch.cat(pooled, 1)
        combined = self.dropout(combined)  # Apply dropout before FC layer
        return self.fc(combined)

# 4. Initialize and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedDropoutTextCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch.to(device))
        loss = criterion(outputs, y_batch.to(device))
        loss.backward()
        optimizer.step()

accuracy, f1 = evaluate(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}, F1: {f1:.4f}')

Test Accuracy: 0.5580, F1: 0.5648
